# Additional training functions

`train` provides a number of extension methods that are added to `Learner` (see below for a list and details), along with three simple callbacks:

- `ShowGraph`
- `GradientClipping`
- `BnFreeze`

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.train import * 

## `Learner` extension methods

These methods are automatically added to all `Learner` objects created after importing this module. They provide convenient access to various callbacks, without requiring them to be manually created.

In [ ]:
show_doc(fit_one_cycle)

#### <a id=fit_one_cycle></a>`fit_one_cycle`
`fit_one_cycle`(`learn`:[`Learner`](/basic_train.html#Learner), `cyc_len`:`int`, `max_lr`:`Union`[`float`, `Collection`[`float`], `slice`]=`slice(None, 0.003, None)`, `moms`:`Tuple`[`float`, `float`]=`(0.95, 0.85)`, `div_factor`:`float`=`25.0`, `pct_start`:`float`=`0.3`, `wd`:`float`=`None`, `kwargs`)


Fits a model following the 1cycle policy <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L11">[source]</a>

In [ ]:
from fastai.callbacks import *

In [ ]:
show_doc(lr_find)

#### <a id=lr_find></a>`lr_find`
`lr_find`(`learn`:[`Learner`](/basic_train.html#Learner), `start_lr`:`float`=`1e-05`, `end_lr`:`float`=`10`, `num_it`:`int`=`100`, `kwargs`:`Any`)


Explore lr from `start_lr` to `end_lr` over `num_it` iterations of `learn` <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L20">[source]</a>

See `OneCycleScheduler` for details.

In [ ]:
show_doc(one_cycle_scheduler)

#### <a id=one_cycle_scheduler></a>`one_cycle_scheduler`
`one_cycle_scheduler`(`lr_max`:`float`, `kwargs`:`Any`) -> [`OneCycleScheduler`](/callbacks.one_cycle.html#OneCycleScheduler)
<a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L8">[source]</a>

See `OneCycleScheduler` for details.

In [ ]:
show_doc(to_fp16)

#### <a id=to_fp16></a>`to_fp16`
`to_fp16`(`learn`:[`Learner`](/basic_train.html#Learner), `loss_scale`:`float`=`512.0`, `flat_master`:`bool`=`False`) -> [`Learner`](/basic_train.html#Learner)


Transforms the learner in FP16 precision <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L26">[source]</a>

See `OneCycleScheduler` for details.

In [ ]:
show_doc(ShowGraph)

## <a id=ShowGraph></a>class `ShowGraph`
`ShowGraph`(`learn`:[`Learner`](/basic_train.html#Learner)) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)


Updates a graph of learner stats and metrics after each epoch <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L44">[source]</a>

[`ShowGraph`](/train.html#ShowGraph)

### Internal callback implementation

In [ ]:
show_doc(ShowGraph.on_epoch_end)

#### <a id=on_epoch_end></a>`on_epoch_end`
`on_epoch_end`(`n_epochs`:`int`, `last_metrics`:`Collection`[`Union`[`Tensor`, `Number`]], `kwargs`) -> `bool`


If we have metrics plot them in our pbar graph <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L46">[source]</a>

In [ ]:
show_doc(GradientClipping)

## <a id=GradientClipping></a>class `GradientClipping`
`GradientClipping`(`learn`:[`Learner`](/basic_train.html#Learner), `clip`:`float`) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)


To do gradient clipping during training. <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L64">[source]</a>

[`GradientClipping`](/train.html#GradientClipping)

### Internal callback implementation

In [ ]:
show_doc(GradientClipping.on_backward_end)

#### <a id=on_backward_end></a>`on_backward_end`
`on_backward_end`(`kwargs`)


Called after backprop but before optimizer step. Useful for true weight decay in AdamW <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L68">[source]</a>

In [ ]:
show_doc(BnFreeze)

## <a id=BnFreeze></a>class `BnFreeze`
`BnFreeze`(`learn`:[`Learner`](/basic_train.html#Learner)) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)


Set all bntypes layers in `learn` to eval() on_epoch_begin <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L57">[source]</a>

[`BnFreeze`](/train.html#BnFreeze)

### Internal callback implementation

In [ ]:
show_doc(BnFreeze.on_epoch_begin)

#### <a id=on_epoch_begin></a>`on_epoch_begin`
`on_epoch_begin`(`kwargs`:`Any`)


Put bn layers in eval mode on epoch_begin <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L59">[source]</a>